# **KawsAI - Un Modelo de IA para Matching de ChambeaYa**

En este notebook se guarda el proceso que se hizo para crear el modelo de IA KawsAI. El objetivo de KawsAI, es hacer el matching con la mejor mype a cada estudiante. Así, el estudiante y la mype pueden conectar para poder trabajar juntos.

### **I. Versión:**
1.0

### **II. Objetivo:**
El objetivo de KawsAI es lograr hacer el mejor match entre estudiantes y mypes. Este match se va a basar en datos que ChambeaYa va a recopilar de ambos lados a traves de su página web. El objetivo principal es lograr indicar a un estudiante cual es el puesto que ofrece una mype que mas coincide con sus conocimientos, y así aumentar mucho las probabilidades de conexión.

### **III. Descripción de los Datasets**
Para la versión 1.0 vamos a usar datasets simulados con la ayuda de la IA de ChatGPT. Se le pidió a ChatGPT simular 2 datasets en forma de tablas y en formato .csv, que son estudiantes_test.csv y mypes_test.csv. Cada una de las columnas de estos datasets contiene información que se va a solicitar a los usuarios ingresar en nuestra plataforma.

##### IIIa. Descripción de columnas de estudiantes:
- **id_estudiante:** ID único del estudiante (string)
- **nombre:** Nombres del estudiante (string)
- **correo:** Correo del estudiante (string)
- **ubicacion:** La ubicación de la residencia del estudiante (string)
- **skill_estrella:** La habilidad mas desarollada o destacada del estudiante (string)
- **motivacion_principal:** Cual es la motivacion mas importante del estudiante en su carrera profesional (string)
- **hobbies:** Hobbies que tiene el estudiante (string)
- **disponibilidad_semanal:** Horas disponibles que tiene el estudiante a la semana para trabajar (int)
- **modalidad_preferida:** Modalidad de trabajo preferida por el estudiante, solo Presencial, Híbrido o Remoto (string)
- **areas_interes:** Áreas profesionales en las que está interesad@ el estudiante. (string)
- **carrera:** La carrera que esta cursando el estudiante (string)
- **ciclo:** El ciclo o semestre en el que se encuentra el estudiante (int)

##### IIIb. Descripción de columnas de mypes:
- **RUC:** RUC de la mype, sirve como identificador (string)
- **nombre_empresa:** El nombre de la mype (string)
- **area_reto:** El area donde la mype esta buscando el puesto u objetivo (string)
- **descripcion_reto:** Es la descripción mas larga de lo que la mype busca y quiere relaizar (string)
- **habilidades_requeridas:** Las habilidades que la mype requiere de los estudiantes que postulen (string)
- **horas_requeridas:** Son las horas semanales que la mype requiere como mínimo. (string)
- **modalidad_proyecto:** Es la modalidad con la que la mype ofreceria empleos. (string)
- **ubicacion:** La ubicación donde se encuentra localizada la mype. (string)

### **IV. Índice**
1. Carga de Librerias

## 1. Carga de Librerias

In [1]:
# Todas las librerias que se deben importar
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
import numpy as np
import spacy
from spacy.lang.es.stop_words import STOP_WORDS as spacy_stopwords
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## 2. Carga de datos en DataFrames

In [19]:
# Ahora leemos los csv y los guardamos en dataframes de pandas
estudiantes = pd.read_csv("../datasets/estudiantes_test.csv", index_col=False)
mypes = pd.read_csv("../datasets/mypes_test.csv", index_col=False)

## 3. Función de Preprocesamiento de Datos

In [ ]:
# Usamos nlp que nos permite realizar el preprocesado mucho mas rápido
nlp = spacy.load("es_core_news_sm")

# La función de pre-procesamiento tiene 4 partes:
# 1) Eliminar simbolos gramaticales
# 2) Tokenización
# 3) Lematización
# 4) Filtrar las stop words (palabras que no aportan contexto importante)
def preprocesar(text):
  
  # Eliminamos todos los signos de puntuación y simbolos gramaticales. Además volvemos el texto minúscula
  text = text.lower()
  text = re.sub(r"[^\w\s]", " ", text)
  text = re.sub(r"\s+", " ", text).strip()

  # Aplicamos la lematización al texto con nlp, que nos provee las palabras ya lematizadas
  doc = nlp(text)

  # Creamos un arreglo para guardar las tokens (palabras del texto separadas)
  tokens = []

  # En un bucle, filtramos todas las tokens que sean puntuacion, que sean stopwords, o que sean puntuacion, espacios, etc.
  for token in doc:
    if token.is_stop or token.is_punct or token.is_space:
      continue
    if token.like_num:
      continue
    # Si el token pasa el filtrado, le aplicamos la lematización
    tokens.append(token.lemma_)
  # Por último, retornamos los tokens como una string que junta todos los datos
  return " ".join(tokens)

## 4. Realizamos el 1er filtrado de Mypes de acuerdo a las características del estudiante 

In [ ]:
# Con esta función, vamos a filtrar las mypes por las columnas que podemos relacionar directamente con el estudiante, como horas_requeridas
# ubicación y modalidad_preferida

# Para llamar esta función, vamos a realizar este filtrado por cada estudiante que quiera ver que mypes le hacen un mejor match
def filtrar_por_columnas(indice_estudiante, estudiantes, mypes):
    estudiante = estudiantes.iloc[indice_estudiante]
    modalidad = estudiante["modalidad_preferida"]
    horas_disponibles = estudiante["disponibilidad_semanal"]

    print(f"{modalidad} {horas_disponibles}")
    # Filtrar por modalidad y horas
    mypes_aux = mypes[
        (mypes["modalidad_proyecto"] == modalidad) &
        (mypes["horas_requeridas"] <= horas_disponibles)
    ]
    return mypes_aux.reset_index(drop=True)


In [22]:
mypes_filtradas = filtrar_por_columnas(6, estudiantes, mypes)

Presencial 25


In [6]:
mypes_filtradas.shape

(665, 10)

# Ahora preprocesamos solo el estudiante y las mypes_filtradas

In [24]:
# Para el estudiante
estudiante = estudiantes.iloc[6]
estudiante["skill_estrella"] = preprocesar(estudiante["skill_estrella"])
estudiante["motivacion_principal"] = preprocesar(estudiante["motivacion_principal"])
estudiante["areas_interes"] = preprocesar(estudiante["areas_interes"])

diseño de material publicitario en adobe illustrator
aprender
fundamentos de diseño gráfico metodologías básicas en diseño gráfico


/tmp/ipykernel_256176/1870823140.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estudiante["skill_estrella"] = preprocesar(estudiante["skill_estrella"])
/tmp/ipykernel_256176/1870823140.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estudiante["motivacion_principal"] = preprocesar(estudiante["motivacion_principal"])
/tmp/ipykernel_256176/1870823140.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estudiante["areas_interes"

In [10]:
estudiante

nombre                                                         Carlos López
correo                                         carlos.lópez21@correo.edu.pe
ubicación                               Trujillo, La Libertad, Víctor Larco
skill_estrella               diseño material publicitario adobe illustrator
motivación_principal                                               aprender
hobbies                                              Voluntariado ambiental
disponibilidad_semanal                                                   25
modalidad_preferida                                              Presencial
areas_interes             fundamento diseño gráfico metodología básico d...
carrera                                                      Diseño Gráfico
ciclo                                                                     3
Name: 6, dtype: object

In [9]:
# Para la mype
mypes_filtradas["area_reto"] = mypes_filtradas["area_reto"].apply(preprocesar)
mypes_filtradas["descripcion_reto"] = mypes_filtradas["descripcion_reto"].apply(preprocesar)
mypes_filtradas["habilidades_requeridas"] = mypes_filtradas["habilidades_requeridas"].apply(preprocesar)
mypes_filtradas["cultura_empresa"] = mypes_filtradas["cultura_empresa"].apply(preprocesar)

In [10]:
mypes_filtradas.head()

,Unnamed: 0,nombre_empresa,RUC,area_reto,descripcion_reto,habilidades_requeridas,cultura_empresa,horas_requeridas,modalidad_proyecto,ubicacion
0,3,Soluciones Marketing Corporación,70925802916,legal,necesitar elaborar política interno claro concisa,redacción legal derecho empresarial análisis c...,empresa joven tecnológico,20,Presencial,"Jr. Amazonas, Huaral, Lima"
1,6,Progreso Marketing SRL,34980195388,marketing,buscar desarrollar estrategia contenido captar...,seo googlar ads marketing digital,organización orientado impacto social,20,Presencial,"Av. Principal, Jaén, Cajamarca"
2,15,Progreso Exportaciones SAC,87188857461,legal,necesitar elaborar política interno claro concisa,excel normativo peruano derecho empresarial an...,startup dinámico colaborativo,25,Presencial,"Jr. San Martín, Moquegua, Moquegua"
3,17,Servicios Exportaciones Corporación,38319868934,recurso humano,mejorar proceso reclutamiento selección,psicología organizacional comunicación efectiv...,empresa familiar enfoque innovación social,25,Presencial,"Av. Principal, Chorrillos, Lima"
4,21,Comercial Diseño SPA,96105252980,diseño,rediseñar página web mejorar experiencia usuario,figma html css adobe illustrator,organización orientado impacto social,25,Presencial,"Av. Libertad, Tarapoto, San Martín"


# 2do Filtro - Creamos los vectores TF-IDF para el subset de mypes

In [15]:
def concatenar_texto_estudiante(estudiantes):
  return (
      str(estudiantes["skill_estrella"]) + " " +
      str(estudiantes["motivacion_principal"]) + " " +
      str(estudiantes["areas_interes"])
  )

def concatenar_texto_mypes(mypes):
  return (
      str(mypes["area_reto"]) + " " +
      str(mypes["descripcion_reto"]) + " " +
      str(mypes["habilidades_requeridas"]) + " " +
      str(mypes["cultura_empresa"])
  )

def calcular_similitud_texto(estudiante, mypes_filtradas):
    # primero concatenamos todo
    texto_estudiante = concatenar_texto_estudiante(estudiante)
    textos_mypes = mypes_filtradas.apply(concatenar_texto_mypes, axis=1)

    # Ahora juntamos el texto estudiante con el texto de las mipes con lista
    corpus = [texto_estudiante] + textos_mypes.tolist()

    # Hacemos la vectorización TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # Ahora hacemos el cosine similarity
    similitudes = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

    # Ahora usamos una columna de similitud
    mypes_filtradas = mypes_filtradas.copy()
    mypes_filtradas["similitud"] = similitudes

    # Ordenamos las mypes_filtradas de mayor a menor
    mypes_filtradas = mypes_filtradas.sort_values(by="similitud", ascending=False).reset_index(drop=True)

    return mypes_filtradas

In [16]:
# 2. Filtrar MYPEs compatibles
mypes_filtradas = filtrar_por_columnas(6, estudiantes, mypes)

# 3. Calcular similitudes
mypes_con_similitud = calcular_similitud_texto(estudiante, mypes_filtradas)

# 4. Ver resultado
mypes_con_similitud[["RUC", "similitud"]]


Presencial 25


,RUC,similitud
0,53206862288,0.391569
1,14894252875,0.388043
2,33820574064,0.381881
3,81227971577,0.381151
4,45831847091,0.370377
...,...,...
660,37965841620,0.000000
661,14366201374,0.000000
662,30804139665,0.000000
663,92231914025,0.000000


In [17]:
estudiantes.iloc[6]

Unnamed: 0                                                                6
nombre                                                         Carlos López
correo                                         carlos.lópez21@correo.edu.pe
ubicacion                                           Jr. Bolognesi, Ica, Ica
skill_estrella            Diseño de material publicitario en Adobe Illus...
motivacion_principal                                               Aprender
hobbies                                              Voluntariado ambiental
disponibilidad_semanal                                                   25
modalidad_preferida                                              Presencial
areas_interes             Fundamentos de Diseño Gráfico, Metodologías bá...
carrera                                                      Diseño Gráfico
ciclo                                                                     3
Name: 6, dtype: object

In [18]:
mypes_filtradas[mypes_filtradas['RUC']==53206862288]

,Unnamed: 0,nombre_empresa,RUC,area_reto,descripcion_reto,habilidades_requeridas,cultura_empresa,horas_requeridas,modalidad_proyecto,ubicacion
239,1725,Estrategias Legal SRL,53206862288,Diseño,Requerimos material gráfico para redes y prese...,"Adobe Illustrator, HTML/CSS",Startup dinámica y colaborativa,20,Presencial,"Av. Progreso, Huánuco, Huánuco"
